# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    P['Expected Return'] = rf + P['Beta'] * mkt_rp
    total_investment = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_investment
    portfolio_expected_return = (P['Expected Return'] * P['Weight']).sum()

    return portfolio_expected_return

In [4]:
print(expected_return(portfolio, 0.02, 0.05))

0.086


Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [5]:
def volatility(P, cor):
    weights = P['Amount Invested'] / P['Amount Invested'].sum()
    
    volatilities = np.array(P['Volatility'])
    n = len(P)
    correlation_matrix = np.ones((n, n)) * cor
    np.fill_diagonal(correlation_matrix, 1)
    variance_matrix = np.outer(volatilities, volatilities) * correlation_matrix
    portfolio_variance = np.dot(weights, np.dot(variance_matrix, weights.T))
    portfolio_volatility = np.sqrt(portfolio_variance)

    return portfolio_volatility

In [6]:
print(volatility(portfolio, -0.2))

0.24083189157584592


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [7]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    sharpe_ratio = mkt_rp / mkt_vol
    expected_return = rf + sharpe_ratio * vol

    return expected_return


In [8]:
print(expected_return2(0.02,0.05,0.16,0.2408))

0.09525


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    output = pd.DataFrame(data = {
        "CUSIP" : [] ,
        "EXP_RETURN" : [],
        "BETA" : [],
    })
    
    return output

In [32]:
def read_data(return_path, factors_path):
    returns = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)
    
    factors['Mkt-RF'] /= 100
    factors['RF'] /= 100
    factors['year'] = factors['date'] // 100
    factors['month'] = factors['date'] % 100
    merged_data = pd.merge(returns, factors, on=['year', 'month'])
    
    return merged_data

def calculate_beta(merged_data):
    merged_data['Excess_RET'] = merged_data['RET'] - merged_data['RF']
    merged_data['Excess_Mkt'] = merged_data['Mkt-RF']
    
    def calc_beta(group):
        cov = group['Excess_RET'].cov(group['Excess_Mkt'])
        var = group['Excess_Mkt'].var()
        beta = cov / var
        return beta
    
    beta_values = merged_data.groupby('CUSIP').apply(calc_beta)
    
    return beta_values

def calculate_expected_return(beta_values, merged_data):
    merged_data['EXP_RETURN'] = merged_data['RF'] + beta_values * merged_data['Excess_Mkt']
    exp_return_values = merged_data.groupby('CUSIP')['EXP_RETURN'].mean()
    
    return exp_return_values

def CAPM_beta(return_path, factors_path):
    merged_data = read_data(return_path, factors_path)
    beta_values = calculate_beta(merged_data)
    exp_return_values = calculate_expected_return(beta_values, merged_data)
    
    output = pd.DataFrame({
        "CUSIP": beta_values.index,
        "EXP_RETURN": exp_return_values.values,
        "BETA": beta_values.values
    })
    
    return output

In [31]:
print(CAPM_beta("monthlycrsp_mod7.csv","F-F_Research_Data_Factors.csv"))

         CUSIP  EXP_RETURN      BETA
0     00081T10         NaN  2.591073
1     00108M10         NaN  0.741057
2     00130H10         NaN  1.457100
3     00163U10         NaN  0.975530
4     00182C10         NaN  1.406085
...        ...         ...       ...
3220   9959910         NaN  0.763107
3221   9972410         NaN  1.280548
3222   9972510         NaN  1.065298
3223   9984910         NaN  1.876418
3224   9985510         NaN  1.166459

[3225 rows x 3 columns]


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [28]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    
    return slope

In [29]:
print(slope(CAPM_beta("monthlycrsp_mod7.csv","F-F_Research_Data_Factors.csv")))

<function slope at 0x000001ADAFC4F880>
